# Inference Time Study

In [1]:
#################################################################################################################
#                                           Initialize Everything                                               #
#################################################################################################################

# Needed imports
import sys
import os
import json
from json_handler import JSONHandler

# Add the new paths separately
new_path1 = "/home/michele/.local/lib/python3.10/site-packages/"
new_path2 = "/home/michele/iac_code/michele_mmdet3d/"
if not new_path1 in sys.path:
    sys.path.insert(1, new_path1)
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)
# Import the fucking wicked class
from mmdet3d.apis import LidarDet3DInferencer

# Initialize the actual inferencer class
inferencer = LidarDet3DInferencer(model='/home/michele/iac_code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py', 
                                  weights='/home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth')

# Read the files in validation list
val_list_txt_file = "/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/ImageSets/val.txt"
with open(val_list_txt_file, 'r') as file:
    val_file_names = [line.strip() for line in file]
print(f"Total validation point_clouds: {len(val_file_names)}")

# Create the list of inputs, suitable for the inferencer
inputs = []
for i, name in enumerate(val_file_names):
    inputs.append(dict(points=("/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/training/velodyne/"+name+".bin")))






out_file = '/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/inference_times.json'
handler = JSONHandler(out_file)






# Remove existing .txt file
out_file = '/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/inference_times.json'
if os.path.exists(out_file):
    os.remove(out_file)
    print("Pre-existing '.json' file removed. Will be created again from scratch")

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Loads checkpoint by local backend from path: /home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth
10/07 23:46:09 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.


/home/michele/iac_code/michele_mmdet3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Total validation point_clouds: 35
Pre-existing '.json' file removed. Will be created again from scratch


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [7]:
#################################################################################################################
#                                          Do the actual inference                                              #
#################################################################################################################

results = []
dictionary = []
dictionary = [{"name": "Alice", "age": 30, "city": "New York"},
              {"name": "Bob", "age": 25, "city": "San Francisco"}]
# Save the list of dictionaries to a JSON file
with open(out_file, "w") as json_file:
    json.dump(dictionary, json_file, indent=4)

for i, input in enumerate(inputs):
    with open(out_file, "r") as json_file:
        loaded_list = json.load(json_file)
        loaded_list.append({"name": "Charlie", "age": (35+i), "city": "Boston"})
    with open(out_file, "w") as json_file:
        json.dump(loaded_list, json_file, indent=4)
    results.append(inferencer(input))

with open(out_file, "r") as json_file:
    loaded_list = json.load(json_file)
for element in loaded_list:
    print(element)

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

{'name': 'Alice', 'age': 30, 'city': 'New York'}
{'name': 'Bob', 'age': 25, 'city': 'San Francisco'}
{'name': 'Charlie', 'age': 35, 'city': 'Boston'}
{'name': 'Charlie', 'age': 36, 'city': 'Boston'}
{'name': 'Charlie', 'age': 37, 'city': 'Boston'}
{'name': 'Charlie', 'age': 38, 'city': 'Boston'}
{'name': 'Charlie', 'age': 39, 'city': 'Boston'}
{'name': 'Charlie', 'age': 40, 'city': 'Boston'}
{'name': 'Charlie', 'age': 41, 'city': 'Boston'}
{'name': 'Charlie', 'age': 42, 'city': 'Boston'}
{'name': 'Charlie', 'age': 43, 'city': 'Boston'}
{'name': 'Charlie', 'age': 44, 'city': 'Boston'}
{'name': 'Charlie', 'age': 45, 'city': 'Boston'}
{'name': 'Charlie', 'age': 46, 'city': 'Boston'}
{'name': 'Charlie', 'age': 47, 'city': 'Boston'}
{'name': 'Charlie', 'age': 48, 'city': 'Boston'}
{'name': 'Charlie', 'age': 49, 'city': 'Boston'}
{'name': 'Charlie', 'age': 50, 'city': 'Boston'}
{'name': 'Charlie', 'age': 51, 'city': 'Boston'}
{'name': 'Charlie', 'age': 52, 'city': 'Boston'}
{'name': 'Charlie

# Inference Time Study

In [ ]:
#################################################################################################################
#                                           Initialize Everything                                               #
#################################################################################################################

# Needed imports
import sys
from mmdet3d.apis import LidarDet3DInferencer
import time
import os
import json

# Add the new paths separately
new_path1 = "/home/michele/.local/lib/python3.10/site-packages/"
new_path2 = "/home/michele/iac_code/michele_mmdet3d/"
if not new_path1 in sys.path:
    sys.path.insert(1, new_path1)
if not new_path2 in sys.path:
    sys.path.insert(1, new_path2)

# Initialize the actual inferencer class
inferencer = LidarDet3DInferencer(model='/home/michele/iac_code/michele_mmdet3d/configs/minerva/CONDENSED_pointpillars_minerva.py', 
                                  weights='/home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth')

# Read the files in validation list
val_list_txt_file = "/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/ImageSets/val.txt"
with open(val_list_txt_file, 'r') as file:
    val_file_names = [line.strip() for line in file]
print(f"Total validation point_clouds: {len(val_file_names)}")

# Create the list of inputs, suitable for the inferencer
inputs = []
for i, name in enumerate(val_file_names):
    inputs.append(dict(points=("/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/training/velodyne/"+name+".bin")))

# Remove existing .txt file
out_file = '/home/michele/iac_code/michele_mmdet3d/data/minerva_polimove/inference_times.json'
if os.path.exists(out_file):
    os.remove(out_file)
    print("Pre-existing '.json' file removed. Will be created again from scratch")

Loads checkpoint by local backend from path: /home/michele/iac_code/michele_mmdet3d/work_dirs/pointpillars_minerva/epoch_120.pth
Total validation point_clouds: 35
Pre-existing '.json' file removed. Will be created again from scratch


/home/michele/iac_code/michele_mmdet3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [ ]:
#################################################################################################################
#                                          Do the actual inference                                              #
#################################################################################################################

results = []
dictionary = []
dictionary = [{"name": "Alice", "age": 30, "city": "New York"},
              {"name": "Bob", "age": 25, "city": "San Francisco"}]
# Save the list of dictionaries to a JSON file
with open(out_file, "w") as json_file:
    json.dump(dictionary, json_file, indent=4)

for i, input in enumerate(inputs):
    with open(out_file, "r") as json_file:
        loaded_list = json.load(json_file)
        loaded_list.append({"name": "Charlie", "age": (35+i), "city": "Boston"})
    with open(out_file, "w") as json_file:
        json.dump(loaded_list, json_file, indent=4)
    results.append(inferencer(input))

with open(out_file, "r") as json_file:
    loaded_list = json.load(json_file)
for element in loaded_list:
    print(element)

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

{'name': 'Alice', 'age': 30, 'city': 'New York'}
{'name': 'Bob', 'age': 25, 'city': 'San Francisco'}
{'name': 'Charlie', 'age': 35, 'city': 'Boston'}
{'name': 'Charlie', 'age': 36, 'city': 'Boston'}
{'name': 'Charlie', 'age': 37, 'city': 'Boston'}
{'name': 'Charlie', 'age': 38, 'city': 'Boston'}
{'name': 'Charlie', 'age': 39, 'city': 'Boston'}
{'name': 'Charlie', 'age': 40, 'city': 'Boston'}
{'name': 'Charlie', 'age': 41, 'city': 'Boston'}
{'name': 'Charlie', 'age': 42, 'city': 'Boston'}
{'name': 'Charlie', 'age': 43, 'city': 'Boston'}
{'name': 'Charlie', 'age': 44, 'city': 'Boston'}
{'name': 'Charlie', 'age': 45, 'city': 'Boston'}
{'name': 'Charlie', 'age': 46, 'city': 'Boston'}
{'name': 'Charlie', 'age': 47, 'city': 'Boston'}
{'name': 'Charlie', 'age': 48, 'city': 'Boston'}
{'name': 'Charlie', 'age': 49, 'city': 'Boston'}
{'name': 'Charlie', 'age': 50, 'city': 'Boston'}
{'name': 'Charlie', 'age': 51, 'city': 'Boston'}
{'name': 'Charlie', 'age': 52, 'city': 'Boston'}
{'name': 'Charlie